In [153]:
import pandas as pd
import numpy as np
import re

In [1]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [2]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [8]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [333]:
def delete_age_time(df, flag):
    
    genders = ['남성', '여성'] 
    days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    for gender in genders:
        gender = flag + gender
        for day in days:
            df[gender+'_'+day+'_유동인구수'] = df.filter(like=gender).filter(like=day).sum(axis=1)
            del_columns = df.filter(like=gender).filter(like=day).filter(like='시간대').columns
            df.drop(del_columns, axis=1, inplace=True)
            
    
    age = flag + '연령대'
    del_columns = df.filter(like=age).columns
    df.drop(del_columns, axis=1, inplace=True)
    
    times = flag + '시간대'
    del_columns = df.filter(like=times).columns
    df.drop(del_columns, axis=1, inplace=True)
    
    for gender in genders:
        gender = flag + gender
        tmp_arr = []
        for day in days[:-2]:
            tmp_arr.append(gender+'_'+day+'_유동인구수')
        df[gender+'_주중_유동인구수'] = df[tmp_arr].sum(axis=1)
        df.drop(tmp_arr, axis=1, inplace=True)
        
        tmp_arr = []
        for day in days[5:]:
            tmp_arr.append(gender+'_'+day+'_유동인구수')
        df[gender+'_주말_유동인구수'] = df[tmp_arr].sum(axis=1)
        df.drop(tmp_arr, axis=1, inplace=True)
        
    tmp_arr = []
    for day in days[5:]:
        tmp_arr.append(day+'_유동인구_수')
    df[flag+'주말_유동인구수'] = df[tmp_arr].sum(axis=1)
    df.drop(tmp_arr, axis=1, inplace=True)
    
    tmp_arr = []
    for day in days[:-2]:
        tmp_arr.append(day+'_유동인구_수')
    df[flag+'주중_유동인구수'] = df[tmp_arr].sum(axis=1)
    df.drop(tmp_arr, axis=1, inplace=True)
    
    return df
    

In [14]:
def cat_df(sg, sg_hl):
    df = load_to_df(sg)
    df0 = df.copy()
    df0["상권/배후지"] = 0
    
    df = load_to_df(sg_hl)
    df_hl = df.copy()
    df_hl["상권/배후지"] = 1
    
    new_columns = list(map( lambda s: s.replace('hl_',''), df_hl.columns ))
    df_hl.columns = new_columns
    
    tmp_df = pd.concat([df0,df_hl])
    return tmp_df.loc[:, (tmp_df != 0).any(axis=0)] 

In [147]:
def pred_selng(df):
    del_columns = df.filter(like='시간대_').filter(like='_매출')
    df.drop(del_columns, axis=1, inplace=True)
    del_columns = df.filter(like='연령대_').filter(like='_매출')
    df.drop(del_columns, axis=1, inplace=True)
    del_columns = df.filter(like='요일_')
    df.drop(del_columns, axis=1, inplace=True)
    
    tmp_dt = {'주중_건당_매출' : ['주중_매출_금액', '주중_매출_건수'],
              '주말_건당_매출' : ['주말_매출_금액', '주말_매출_건수'],
              '남성_건당_매출' : ['남성_매출_금액', '남성_매출_건수'],
              '여성_건당_매출' : ['여성_매출_금액', '여성_매출_건수']}

    for key, value in tmp_dt.items():
        join_grouped[key]=join_grouped[value[0]]/join_grouped[value[1]]
    
    return join_grouped

In [19]:
df = load_to_df('상권-영역')

df00 = df.copy()
len(df00['상권_코드'].unique())

1496

In [20]:
#### 상권/배후지 집객시설 (02, 04)

In [21]:
df_02_04 = cat_df('상권-집객시설', '상권배후지-집객시설')
df_02_04

,기준_년_분기_코드,상권_코드,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,...,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,버스_터미널_수,지하철_역_수,버스_정거장_수,상권/배후지
0,2019_2,1000953,14,0,0,0,0,3,0,0,...,0,0,0,0,2,0,0,0,1,0
1,2019_2,1001361,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2019_2,1001387,2,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,2019_2,1001366,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2019_2,1001308,4,0,0,0,0,0,1,0,...,0,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_1,1000776,154,6,4,0,1,17,0,2,...,0,0,1,0,33,0,0,1,25,1
2016,2019_1,1000958,168,9,13,0,6,15,1,1,...,0,0,10,0,16,0,0,2,22,1
2017,2019_1,1000689,116,4,4,0,1,9,2,1,...,0,0,1,0,5,0,0,1,21,1
2018,2019_1,1000774,97,4,1,1,1,6,0,1,...,0,0,1,0,16,0,0,1,26,1


In [28]:
# 6 개 데이터 통합하기
# 01 상권-추정유동인구
# 03 상권배후지-추정유동인구
# 06 상권_상주인구
# 05 상권배후지-상주인구
# 12 상권-직장인구
# 14 상권배후지-직장인구
# ['기준_년_분기_코드', '상권_코드', '총_인구_수', '남성_인구_수', '여성_인구_수', '상권/배후지', "유동/상주/직장인구"]

In [31]:
pred_sel_df = pred_selng(df_11_19)

In [32]:
len(pred_sel_df.index)/2

1487.0

In [33]:
pred_sel_df

서비스_업종_코드    점포수      당월_매출_금액  당월_매출_건수  주중_매출_비율  \
기준_년_분기_코드 상권_코드                                                         
2019_1     1000001         55    700   27842205529   1613367      3946   
           1000002         68   1331   54426986602   3254283      5064   
           1000003         74   2301  104171705340   4767294      5893   
           1000004         53    300   10007101199   1202019      4084   
           1000005         43    188    7658787640    396405      2702   
...                       ...    ...           ...       ...       ...   
2019_2     1001492         66   8672  426178128873  17383753      5432   
           1001493         61  16757  316842033367   7499782      4324   
           1001494         66   6331  197565922782   6654957      5189   
           1001495         62   2899  235789472396   8887262      4484   
           1001496         48   1289  231358202820   5966946      3462   

                    주말_매출_비율  남성_매출_비율  여성_매출_비율      주중_매출_금액      주말_매출_금액  \
기준_년_분기_코드 상권_코드                                                               
2019_1     1000001      1560      2322      3182   20160815917    7681389612   
           1000002      1740      3630      3175   39606245958   14820740644   
           1000003      1510      3914      3494   75872912026   28298793314   
           1000004      1220      2940      2362    7506584456    2500516743   
           1000005      1601      1900      2400    4641277982    3017509658   
...                      ...       ...       ...           ...           ...   
2019_2     1001492      1176      3157      3447  335637620571   90540508302   
           1001493      1781      3002      3101  199885639936  116805697694   
           1001494      1415      3663      2941  150152311519   47413611263   
           1001495      1723      3175      3030  142390475582   93398996814   
           1001496      1339      2035      2771  170819748283   60538454537   

                    ...      여성_매출_금액  주중_매출_건수  주말_매출_건수  남성_매출_건수  여성_매출_건수  \
기준_년_분기_코드 상권_코드    ...                                                         
2019_1     1000001  ...   13208408646   1195511    417856    772694    768284   
           1000002  ...   22353942043   2373646    880637   1801691   1346942   
           1000003  ...   44041451121   3521208   1246086   2442592   2097908   
           1000004  ...    4200961849    936648    265371    704243    495807   
           1000005  ...    3711674999    220313    176092    213161    170088   
...                 ...           ...       ...       ...       ...       ...   
2019_2     1001492  ...  215584639339  13825316   3558437   7509765   8873858   
           1001493  ...  178278100510   5083674   2416108   3270139   4125299   
           1001494  ...   84389882606   5021371   1633586   3488987   2858615   
           1001495  ...  103892103155   5582428   3304834   4199024   4456242   
           1001496  ...  139174352160   4264541   1702405   2158036   3551562   

                    상권/배후지      주중_건당_매출      주말_건당_매출      남성_건당_매출  \
기준_년_분기_코드 상권_코드                                                       
2019_1     1000001      32  16863.764463  18382.863025  14368.261209   
           1000002      42  16685.826765  16829.568419  14434.674043   
           1000003      41  21547.409874  22710.144656  19545.052525   
           1000004      29   8014.306822   9422.720429   8097.207355   
           1000005      26  21066.745866  17135.983793  15344.222714   
...                    ...           ...           ...           ...   
2019_2     1001492       0  24277.030671  25443.898066  21008.138449   
           1001493       0  39319.130207  48344.568080  39461.092028   
           1001494       0  29902.652387  29024.251716  26184.603076   
           1001495       0  25506.907672  28261.327744  27327.029730   
           1001496       0  40055.834446  35560.547894  36206.005983   

                        여

# 다이나믹 기준 보기

In [120]:
ch_df = load_to_df('상권-상권변화지표')
ch_df['평균_폐업_비율'] = ch_df['폐업_영업_개월_평균']/ch_df['서울_폐업_영업_개월_평균']
ch_df['평균_운영_비율'] = ch_df['운영_영업_개월_평균']/ch_df['서울_운영_영업_개월_평균']

In [121]:
ch_df[(ch_df['기준_년_분기_코드']=='2019_1')
      &(ch_df['상권_변화_지표_명'] == '정체')][['상권_코드', '상권_변화_지표', '상권_변화_지표_명', '평균_운영_비율', '평균_폐업_비율']]

,상권_코드,상권_변화_지표,상권_변화_지표_명,평균_운영_비율,평균_폐업_비율
5598,1001119,HH,정체,1.046729,1.132075
5601,1001116,HH,정체,1.130841,1.018868
5604,1001113,HH,정체,1.065421,1.433962
5614,1001103,HH,정체,1.037383,1.132075
5615,1001102,HH,정체,1.196262,1.132075
...,...,...,...,...,...
7533,1000254,HH,정체,1.121495,1.075472
7537,1000250,HH,정체,1.084112,1.094340
7552,1000235,HH,정체,1.121495,1.056604
7559,1000228,HH,정체,1.074766,1.056604


In [40]:
pd.get_dummies(ch_df['상권_변화_지표'])

,HH,HL,LH,LL
0,1,0,0,0
1,0,0,0,1
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
37395,0,0,0,1
37396,0,0,0,1
37397,0,0,0,1
37398,0,0,0,1


In [44]:
def unique_sg_cd(sg, sg_hl):
    a_df = load_to_df(sg)
    b_df = load_to_df(sg_hl)
    
    len(set(a_df['상권_코드'].unique())-set(b_df['상권_코드'].unique()))
    a_count = len(a_df['상권_코드'].unique())
    b_count = len(b_df['상권_코드'].unique())
    count = len(a_df['상권_코드'].unique()) - len(set(a_df['상권_코드'].unique())-set(b_df['상권_코드'].unique()))
    del a_df, b_df
    print(a_count, b_count, count, sep=' : ')

In [46]:
# pop
unique_sg_cd('상권-추정유동인구', '상권배후지-추정유동인구')
unique_sg_cd('상권-상주인구', '상권배후지-상주인구')
unique_sg_cd('상권-직장인구', '상권배후지-직장인구')

# selling
unique_sg_cd('상권-추정매출', '상권배후지-추정매출')

# place
unique_sg_cd('상권-집객시설', '상권배후지-집객시설')
unique_sg_cd('상권-아파트', '상권배후지-아파트')
unique_sg_cd('상권-점포', '상권배후지-점포')


1495 : 1010 : 1010
1481 : 1010 : 1010
1494 : 1010 : 1010
1488 : 1010 : 1010
1462 : 1010 : 1008
1322 : 1009 : 990
1496 : 1010 : 1010


In [47]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    del a_df, b_df
    
    return all_df.dropna(axis=0)

In [51]:
load_to_df('상권-추정매출')

,기준_년_분기_코드,상권_코드,서비스_업종_코드,서비스_업종_코드_명,점포수,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,...,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,2019_2,1001496,CS300043,전자상거래업,7,20931734,13998,71,29,8,...,3015,1480,8977,5016,526,4829,3886,2749,1507,496
1,2019_2,1001496,CS300035,인테리어,4,2000367097,48419,69,31,12,...,16632,1478,11553,36497,0,5200,20888,16005,2604,3353
2,2019_2,1001496,CS300032,가전제품,8,2301647043,11720,68,33,25,...,3209,288,4776,6524,180,2866,2432,2297,2213,1309
3,2019_2,1001496,CS300028,화초,6,1356873015,33215,71,29,12,...,15926,1636,14364,16873,256,10677,12052,4323,2612,1314
4,2019_2,1001496,CS300027,섬유제품,7,2130125149,62318,62,38,11,...,29713,4926,10911,51407,2258,30634,11715,8385,7121,2205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95986,2019_1,1000015,CS100004,양식음식점,4,227301115,9913,65,35,10,...,4509,470,4370,5468,273,4258,1937,1889,1040,439
95987,2019_1,1000015,CS100003,일식음식점,4,297895879,15555,83,17,16,...,5949,101,6454,8125,269,5448,3904,2745,1526,687
95988,2019_1,1000015,CS100002,중식음식점,2,30878767,2270,91,9,15,...,412,0,1196,662,12,401,586,552,255,52
95989,2019_1,1000015,CS100001,한식음식점,26,2340790231,75692,85,15,15,...,27493,1750,28781,38980,475,19942,18079,13139,10540,5586


In [125]:
def sg_OneHotEncode(dest_df):
    # 상권 변화 지표 합치기
    sg_df = load_to_df('상권-상권변화지표')
    sg_df = sg_df[(sg_df['기준_년_분기_코드']=='2019_1')|(sg_df['기준_년_분기_코드']=='2019_2')]
    target_df = pd.merge(dest_df, sg_df[['기준_년_분기_코드', '상권_코드', '상권_변화_지표']],on=['기준_년_분기_코드', '상권_코드'])
    # 상권 지표 OneHotEncode 변경
    one_hot_encoding = pd.get_dummies(target_df['상권_변화_지표'])
    target_df = pd.concat([pd.DataFrame(target_df), one_hot_encoding],axis=1)

    return target_df.drop('상권_변화_지표',axis=1)


In [127]:
#### 상권/배후지 집객시설 (02, 04)
jip_df = multi_columns_merging('상권-집객시설', '상권배후지-집객시설')
jip2_df = sg_OneHotEncode(jip_df)

In [128]:
#### 상권/배후지 아파트 (07, 16)
apart_df = multi_columns_merging('상권-아파트', '상권배후지-아파트')
apart2_df = sg_OneHotEncode(apart_df) # 아파트
columns_1 = apart2_df.filter(like='아파트_면적_').columns
columns_2 = apart2_df.filter(like='아파트_가격_').columns
apart2_df.drop(columns_1, axis=1, inplace=True)
apart2_df.drop(columns_2, axis=1, inplace=True)

In [129]:
apart2_df

,기준_년_분기_코드,상권_코드,아파트_단지_수,아파트_평균_면적,아파트_평균_시가,hl_아파트_단지_수,hl_아파트_평균_면적,hl_아파트_평균_시가,HH,HL,LH,LL
0,2019_1,1000952,71,46,160132655,465.0,54.0,195104843.0,0,0,0,1
1,2019_2,1000951,113,49,178377150,587.0,51.0,177578507.0,0,0,0,1
2,2019_1,1000951,113,49,178377150,587.0,51.0,177578507.0,0,0,0,1
3,2019_2,1000950,119,47,167313286,391.0,52.0,187299414.0,0,0,0,1
4,2019_1,1000950,119,47,167313286,391.0,52.0,187299414.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1975,2019_1,1000134,43,50,151913640,213.0,52.0,148594250.0,1,0,0,0
1976,2019_2,1000133,57,51,150264017,431.0,55.0,154300553.0,1,0,0,0
1977,2019_1,1000133,57,51,150264017,431.0,55.0,154300553.0,1,0,0,0
1978,2019_2,1000132,76,53,142053286,292.0,54.0,164267098.0,0,0,1,0


In [130]:
def sg_svc_category_merge(df):
    grouped = df.groupby(['기준_년_분기_코드','상권_코드'])
    grouped_part1 = grouped.count() # 각 상권별 서비스 업종 코드수    
    grouped_part2 = grouped.sum() # 각 상권별 서비스 업종 분류수    
    join_grouped = grouped_part1.iloc[:,:1].join(grouped_part2, on=['기준_년_분기_코드', '상권_코드'], how='inner')
    return join_grouped

In [141]:
def multi_columns_merging_svc_cd(sg, sg_hl):
    jumpo_df = load_to_df(sg)
    hl_jumpo_df = load_to_df(sg_hl)
    
    jumpo_df = sg_svc_category_merge(jumpo_df)
    hl_jumpo_df = sg_svc_category_merge(hl_jumpo_df)
    
    hl_jumpo_df.rename(columns={'서비스_업종_코드':'hl_서비스_업종_코드'})
    
    all_df = pd.merge(left=jumpo_df, right=hl_jumpo_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    del jumpo_df, hl_jumpo_df
    
    return all_df.dropna(axis=0)

In [143]:
jumpo_df = multi_columns_merging_svc_cd('상권-점포', '상권배후지-점포')
sg_OneHotEncode(jumpo_df)

,기준_년_분기_코드,상권_코드,서비스_업종_코드_x,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,...,hl_유사_업종_점포_수,hl_개업_율,hl_개업_점포_수,hl_폐업_률,hl_폐업_점포_수,hl_프랜차이즈_점포_수,HH,HL,LH,LL
0,2019_1,1000001,71,233,244,44,4,41,4,11,...,741.0,36.0,13.0,67.0,15.0,54.0,0,0,1,0
1,2019_1,1000002,80,142,149,211,3,134,11,7,...,1467.0,202.0,39.0,246.0,61.0,95.0,0,1,0,0
2,2019_1,1000003,81,322,336,211,9,189,8,14,...,3268.0,182.0,73.0,297.0,86.0,126.0,1,0,0,0
3,2019_1,1000004,56,97,109,14,2,31,3,12,...,290.0,72.0,7.0,75.0,10.0,32.0,0,0,1,0
4,2019_1,1000005,58,77,81,33,1,213,5,4,...,198.0,190.0,7.0,108.0,9.0,8.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_2,1001006,77,144,164,150,9,102,6,20,...,825.0,285.0,26.0,508.0,31.0,72.0,0,0,1,0
2016,2019_2,1001007,78,169,181,195,8,265,9,12,...,516.0,202.0,17.0,142.0,16.0,40.0,0,0,1,0
2017,2019_2,1001008,80,206,220,109,10,35,5,14,...,1905.0,266.0,45.0,245.0,57.0,141.0,0,0,0,1
2018,2019_2,1001009,85,222,234,161,3,89,6,12,...,1596.0,325.0,48.0,434.0,44.0,133.0,0,1,0,0


In [144]:
#### 상권/배후지 추정매출 (11, 19)

In [197]:
pred_sel_df = multi_columns_merging_svc_cd('상권-추정매출', '상권배후지-추정매출')
pred_sel_df = sg_OneHotEncode(pred_sel_df)

In [198]:
pred_sel_df.columns.to_list()
yo_columns = pred_sel_df.filter(like='요일_').columns
si_columns = pred_sel_df.filter(like='시간대_').columns
age_columns = pred_sel_df.filter(like='연령대_').columns
# yo_columns = list(map( lambda s: s.replace('\n',''), yo_columns ))
# pred_sel_df.drop(yo_columns.to_list(), axis=1, inplace=True)
# pred_sel_df.drop(si_columns.to_list(), axis=1, inplace=True)
# pred_sel_df.drop(age_columns.to_list(), axis=1, inplace=True)
pred_sel_df.drop(yo_columns, axis=1, inplace=True)
pred_sel_df.drop(si_columns, axis=1, inplace=True)
pred_sel_df.drop(age_columns, axis=1, inplace=True)

# yo_columns.to_list()

In [215]:
pred_sel_df.columns.to_list()
pred_sel_df.rename(columns={'서비스_업종_코드_x':'서비스_업종_코드', '서비스_업종_코드_y':'hl_서비스_업종_코드'}, inplace=True)
pred_sel_df[['주중_매출_금액','주말_매출_금액']]

,주중_매출_금액,주말_매출_금액
0,3180426031,1447986596
1,6367720225,2582044270
2,7245208634,2178388583
3,2554418559,846357318
4,1505935110,1005049071
...,...,...
1633,850945539,352797378
1634,4731102004,2306048337
1635,3402354976,913485443
1636,1812356236,888519921


In [216]:
pred_sel_df['주중_매출_비율'] = pred_sel_df['주중_매출_금액'] / (pred_sel_df['주중_매출_금액'] + pred_sel_df['주말_매출_금액'])*100
pred_sel_df['주말_매출_비율'] = pred_sel_df['주말_매출_금액'] / (pred_sel_df['주중_매출_금액'] + pred_sel_df['주말_매출_금액'])*100
pred_sel_df['남성_매출_비율'] = pred_sel_df['남성_매출_금액'] / (pred_sel_df['남성_매출_금액'] + pred_sel_df['여성_매출_금액'])*100
pred_sel_df['여성_매출_비율'] = pred_sel_df['여성_매출_금액'] / (pred_sel_df['남성_매출_금액'] + pred_sel_df['여성_매출_금액'])*100

pred_sel_df['hl_주중_매출_비율'] = pred_sel_df['hl_주중_매출_금액'] / (pred_sel_df['hl_주중_매출_금액'] + pred_sel_df['hl_주말_매출_금액'])*100
pred_sel_df['hl_주말_매출_비율'] = pred_sel_df['hl_주말_매출_금액'] / (pred_sel_df['hl_주중_매출_금액'] + pred_sel_df['hl_주말_매출_금액'])*100
pred_sel_df['hl_남성_매출_비율'] = pred_sel_df['hl_남성_매출_금액'] / (pred_sel_df['hl_남성_매출_금액'] + pred_sel_df['hl_여성_매출_금액'])*100
pred_sel_df['hl_여성_매출_비율'] = pred_sel_df['hl_여성_매출_금액'] / (pred_sel_df['hl_남성_매출_금액'] + pred_sel_df['hl_여성_매출_금액'])*100
pred_sel_df[['주중_매출_비율','주말_매출_비율', 'hl_주중_매출_비율','hl_주말_매출_비율']]

,주중_매출_비율,주말_매출_비율,hl_주중_매출_비율,hl_주말_매출_비율
0,68.715266,31.284734,73.147848,26.852152
1,71.149584,28.850416,73.088294,26.911706
2,76.883683,23.116317,72.431740,27.568260
3,75.112817,24.887183,74.960975,25.039025
4,59.973899,40.026101,60.906421,39.093579
...,...,...,...,...
1633,70.691634,29.308366,75.142628,24.857372
1634,67.230367,32.769633,67.326315,32.673685
1635,78.834124,21.165876,80.801657,19.198343
1636,67.102530,32.897470,75.521138,24.478862


In [219]:
pred_sel_df

,기준_년_분기_코드,상권_코드,서비스_업종_코드,점포수,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,남성_매출_비율,여성_매출_비율,...,hl_남성_매출_금액,hl_여성_매출_금액,hl_주중_매출_건수,hl_주말_매출_건수,hl_남성_매출_건수,hl_여성_매출_건수,HH,HL,LH,LL
0,2019_1,1000001,23,174,4628412627,298119,68.715266,31.284734,40.179135,59.820865,...,9.444083e+09,1.073961e+10,980879.0,334369.0,641407.0,614200.0,0,0,1,0
1,2019_1,1000002,26,132,8949764495,343625,71.149584,28.850416,41.353139,58.646861,...,2.283884e+10,1.786112e+10,2136886.0,773772.0,1640615.0,1171936.0,0,1,0,0
2,2019_1,1000003,33,208,9423597217,358803,76.883683,23.116317,50.488938,49.511062,...,4.402554e+10,4.039836e+10,3250855.0,1157636.0,2242472.0,1961696.0,1,0,0,0
3,2019_1,1000004,24,99,3400775877,378584,75.112817,24.887183,58.577910,41.422090,...,3.726967e+09,2.804076e+09,650493.0,172942.0,483463.0,338517.0,0,0,1,0
4,2019_1,1000005,17,55,2510984181,89060,59.973899,40.026101,46.854058,53.145942,...,2.191419e+09,2.487358e+09,169050.0,138295.0,171798.0,125619.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,2019_2,1000624,22,71,1203742917,74987,70.691634,29.308366,49.818500,50.181500,...,9.274562e+09,8.046595e+09,641582.0,234953.0,481503.0,385928.0,0,1,0,0
1634,2019_2,1000625,21,108,7037150341,359324,67.230367,32.769633,67.899134,32.100866,...,2.787033e+10,2.176593e+10,1647924.0,669297.0,1208029.0,1054515.0,1,0,0,0
1635,2019_2,1000626,27,80,4315840419,80982,78.834124,21.165876,42.166589,57.833411,...,1.855149e+10,1.630533e+10,1825542.0,330052.0,1181231.0,889820.0,0,0,0,1
1636,2019_2,1000627,31,151,2700876157,168910,67.102530,32.897470,60.917854,39.082146,...,2.706800e+10,2.634830e+10,2261188.0,800906.0,1552983.0,1465445.0,0,0,0,1


In [224]:
#### 상권/배후지 상주인구 (06, 05)

In [234]:
live_pop_df = multi_columns_merging('상권-상주인구', '상권배후지-상주인구')
live_pop_df = sg_OneHotEncode(live_pop_df)
delete_columns = live_pop_df.filter(like='연령대_').columns
live_pop_df.drop(delete_columns, axis=1, inplace=True)

In [235]:
live_pop_df

,기준_년_분기_코드,상권_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,hl_총_가구_수,hl_아파트_가구_수,hl_비_아파트_가구_수,HH,HL,LH,LL
0,2019_2,1000979,3269,1651,1618,1656,122,1534,22129.0,11025.0,11104.0,9670.0,2935.0,6735.0,0,0,0,1
1,2019_2,1000978,3651,1791,1860,1599,333,1266,21533.0,10838.0,10695.0,10181.0,1087.0,9094.0,0,0,0,1
2,2019_2,1000977,2767,1379,1388,1274,114,1160,23136.0,11442.0,11694.0,9377.0,3085.0,6292.0,0,0,0,1
3,2019_2,1000976,2914,1498,1416,1274,0,1274,13231.0,6793.0,6438.0,4666.0,767.0,3899.0,0,0,0,1
4,2019_2,1000975,1694,915,779,821,0,821,4617.0,2456.0,2161.0,2112.0,0.0,2112.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_1,1000005,1694,826,868,725,0,725,6102.0,2958.0,3144.0,2607.0,8.0,2599.0,0,0,0,1
2016,2019_1,1000004,1453,711,742,884,0,884,9137.0,4521.0,4616.0,4910.0,257.0,4653.0,0,0,1,0
2017,2019_1,1000003,1214,682,532,805,0,805,3512.0,2033.0,1479.0,2394.0,150.0,2244.0,1,0,0,0
2018,2019_1,1000002,841,383,458,775,8,767,11047.0,5624.0,5423.0,6257.0,2285.0,3972.0,0,1,0,0


In [236]:
#### 상권/배후지 직장인구 (12, 14)

In [243]:
job_pop_df = multi_columns_merging('상권-직장인구', '상권배후지-직장인구')
job_pop_df = sg_OneHotEncode(job_pop_df)
delete_columns = job_pop_df.filter(like='연령대_').columns
job_pop_df.drop(delete_columns, axis=1, inplace=True)

In [244]:
job_pop_df

,기준_년_분기_코드,상권_코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수,HH,HL,LH,LL
0,2019_2,1000918,204,93,111,14517.0,7537.0,6980.0,0,0,0,1
1,2019_2,1000917,837,420,417,15288.0,9326.0,5962.0,0,0,0,1
2,2019_2,1000916,1383,854,529,26050.0,14721.0,11329.0,0,0,0,1
3,2019_2,1000915,477,278,199,11412.0,7492.0,3920.0,0,0,0,1
4,2019_2,1000914,4389,2637,1752,26569.0,16745.0,9824.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_1,1000005,1080,692,388,1820.0,994.0,826.0,0,0,0,1
2016,2019_1,1000004,104,58,46,2196.0,1194.0,1002.0,0,0,1,0
2017,2019_1,1000003,4822,2188,2634,44890.0,32318.0,12572.0,1,0,0,0
2018,2019_1,1000002,486,254,232,13270.0,6153.0,7117.0,0,1,0,0


In [245]:
#### 상권/배후지 추정유동인구 (01, 03)

In [335]:
pred_pop_df = load_to_df('상권-추정유동인구')
tmp_df = delete_age_time(pred_pop_df, flag='')

In [338]:
tmp_df
pred_pop_df = load_to_df('상권배후지-추정유동인구')
tmp_df2 = delete_age_time(pred_pop_df, flag='hl_')

KeyError: "None of [Index(['토요일_유동인구_수', '일요일_유동인구_수'], dtype='object')] are in the [columns]"